# 随机抽样

In [50]:
import tensorflow as tf
import numpy as np

In [51]:
a = np.random.random_sample((3, 10,5))
a = np.reshape(a,[3,50])

In [52]:
a.shape

(3, 50)

In [53]:
output = np.reshape(a[:,0:50].astype(float), (-1, 5))

In [54]:
outrank = np.argmax(output, axis = 1)
outrank

array([4, 4, 1, 3, 1, 3, 2, 0, 0, 0, 4, 4, 0, 4, 4, 4, 4, 1, 2, 2, 2, 1,
       2, 0, 3, 3, 0, 2, 0, 0], dtype=int64)

In [84]:
output = np.reshape(a[:,0:50].astype(float), (-1, 10, 5))[:,:,0:4]
output.shape  # [bs,10,4]

(3, 10, 4)

In [144]:
def sample_one(table):
    random_shifts = np.random.random(table.shape)
    random_shifts /= random_shifts.sum(axis = 1)[:,np.newaxis]
    result = []
    print('sample_one,table_shape',table.shape)
    for i in range(table.shape[0]):
        table[i] /= table[i].sum()
        shifted_probabilities = random_shifts[i] - table[i]
        
        l = np.argpartition(shifted_probabilities, 1, axis=0)[0].astype(np.int64)
        table[i:,l] = 0
        result.append(l)
        print('sample_one',table,l)
    return result

In [145]:
def sample_set(table, n):
    result = []
    print('table-shape',table.shape)  # [4,10]
    for i in range(n):
        r = sample_one(table.copy())  # [1,4]
        print('sample_set,r',r)
        if not r in result:
            result.append(r)  # [20, 4]
    return np.array(result)

In [146]:
def is_comb(result):
    return (np.sum(result) == 6)
def is_right(result):
    return (result[0] == 0 and result[1] == 1 and result[2] == 2 and result[3] == 3)

In [174]:
def sample_and_test(prob_table, temperature, k=20):
    count = 0
    set_count = 0
    right = 0
    comb = 0
    res = []
    for i in range(prob_table.shape[0]):
        table = prob_table[i]  # [70,4]
        table = np.transpose(table)  # [4,70]
        table = np.exp(table * temperature)
        result = sample_set(table, k)  # [20,5-1]
        res.append(np.array(result))
#         for r in result:
#             count += 1
#             if (is_comb(r)):
#                 comb += 1
#                 if (is_right(r)):
#                     right += 1
#         set_count += 1
    #print(right/float(set_count), comb/float(count), set_count, count)
    print(res)
    return np.array(res)

In [183]:
k_list = sample_and_test(output, 0.4)

table-shape (4, 10)
sample_one,table_shape (4, 10)
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663536
  0.08883014 0.11984099 0.11865811 0.        ]
 [1.36885184 1.06306624 1.36478918 1.10353117 1.44675057 1.13381408
  1.00779503 1.18471609 1.08729835 0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366877 1.00831026 1.10047868 0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682761 1.09413342 1.3955695
  1.16789658 1.35496672 1.05787337 0.        ]] 9
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663536
  0.08883014 0.11984099 0.11865811 0.        ]
 [0.12720947 0.09879235 0.12683192 0.10255282 0.13444872 0.
  0.09365592 0.11009746 0.10104428 0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 0.
  1.20366877 1.00831026 1.10047868 0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682761 1.09413342 0.
  1.16789658 1.35496672 1.05787337 0.        ]] 5
sample_one [[0.08126751 0.

  1.377913   1.2541098  1.11613696 1.28286582]] 3
sample_one [[0.0788487  0.1003359  0.09693574 0.         0.09809729 0.09686096
  0.10834219 0.09004729 0.11404668 0.1068181 ]
 [0.12976825 0.12826599 0.         0.         0.10060966 0.10401279
  0.09424987 0.12096431 0.10545387 0.09621867]
 [1.48415619 1.26051908 0.         0.         1.39207674 1.18648197
  1.3736276  1.44615824 1.02053348 1.18216309]
 [1.27431691 1.08353515 0.         0.         1.4813168  1.2574542
  1.377913   1.2541098  1.11613696 1.28286582]] 2
sample_one [[0.0788487  0.1003359  0.09693574 0.         0.09809729 0.09686096
  0.10834219 0.09004729 0.11404668 0.1068181 ]
 [0.12976825 0.12826599 0.         0.         0.10060966 0.10401279
  0.09424987 0.12096431 0.10545387 0.09621867]
 [0.         0.12183971 0.         0.         0.13455586 0.1146834
  0.13277259 0.13978329 0.09864309 0.11426595]
 [0.         1.08353515 0.         0.         1.4813168  1.2574542
  1.377913   1.2541098  1.11613696 1.28286582]] 0
sampl

In [184]:
k_list.shape

(3, 20, 4)

In [185]:
output

array([[[0.01763196, 0.78493079, 0.44077008, 0.31330443],
        [0.28858395, 0.15289352, 0.08271253, 0.19082091],
        [0.45205545, 0.77749992, 0.55354924, 0.72441754],
        [0.67238919, 0.24628798, 0.25232916, 0.87105074],
        [0.15843012, 0.92330014, 0.29877641, 0.22490664],
        [0.45062772, 0.31396809, 0.05024206, 0.83325644],
        [0.24008112, 0.01941202, 0.46343551, 0.38801083],
        [0.98868054, 0.4237579 , 0.0206898 , 0.75944223],
        [0.96388179, 0.20924011, 0.23936311, 0.1406516 ],
        [0.90577862, 0.22551548, 0.43431037, 0.09053341]],

       [[0.2577584 , 0.37867233, 0.10181423, 0.57026609],
        [0.39858533, 0.0453737 , 0.29448609, 0.81484825],
        [0.87736571, 0.77326876, 0.01139093, 0.36366011],
        [0.45654379, 0.55435786, 0.47245195, 0.67826014],
        [0.16794107, 0.43807598, 0.31894522, 0.04191488],
        [0.40214315, 0.08277325, 0.62191263, 0.05529954],
        [0.27910872, 0.20666079, 0.16087407, 0.06769208],
        [0.5

In [188]:
emb = tf.nn.embedding_lookup(output,k_list)
emb

<tf.Tensor 'embedding_lookup_1/Identity:0' shape=(3, 20, 4, 10, 4) dtype=float64>

In [187]:
emb_lookup = sess.run(emb)

InvalidArgumentError: indices[0,0,0] = 9 is not in [0, 3)
	 [[node embedding_lookup (defined at d:\python\anaconda3\envs\tf15\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'embedding_lookup':
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\python\anaconda3\envs\tf15\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\ipykernel\kernelapp.py", line 597, in start
    self.io_loop.start()
  File "d:\python\anaconda3\envs\tf15\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "d:\pyt

In [176]:
def tensor_sample(prob_table_tensor,temperature,k):
    res=tf.py_func(sample_and_test,[prob_table_tensor,temperature,k],tf.int64)
    return res

In [177]:
sess = tf.Session()

In [178]:
ts=tensor_sample(tf.convert_to_tensor(output),tf.convert_to_tensor(0.4),tf.convert_to_tensor(20))

In [179]:
ts=tensor_sample(output,0.4,3)

In [180]:
qweq= sess.run(ts)
qweq

table-shape (4, 10)
sample_one,table_shape (4, 10)
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [1.36885185 1.06306624 1.36478918 1.10353117 1.44675058 1.13381408
  1.00779503 1.1847161  1.08729835 0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 1.10047868 0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 1.05787337 0.        ]] 9
sample_one [[0.08126751 0.09057037 0.09669056 0.10559904 0.08597578 0.09663535
  0.08883014 0.11984099 0.11865811 0.        ]
 [0.12720947 0.09879235 0.12683192 0.10255282 0.13444872 0.10536706
  0.09365592 0.11009746 0.         0.        ]
 [1.19280543 1.03363841 1.24784704 1.10620105 1.12694515 1.02030012
  1.20366878 1.00831026 0.         0.        ]
 [1.13351313 1.07931692 1.33611616 1.41682762 1.09413343 1.39556951
  1.16789658 1.35496673 0.         0.        ]] 8
s

array([[[9, 8, 2, 5],
        [8, 5, 2, 1],
        [9, 4, 0, 2]],

       [[6, 5, 8, 3],
        [2, 6, 3, 0],
        [7, 6, 8, 3]],

       [[8, 1, 4, 0],
        [3, 7, 2, 5],
        [0, 1, 3, 6]]], dtype=int64)

In [182]:
qweq.shape

(3, 3, 4)